In [1]:
import pandas as pd
import subprocess

In [3]:
#DisGeNet
# curated only, default parameters
PD_SNPs_DisGeNET=pd.read_csv("C0030567_disease_vda_summary_CURATED.tsv",delimiter = '\t')#321 Parkinson 
PD_snps_ids_disgenet=PD_SNPs_DisGeNET["Variant"].tolist()

321


In [12]:
def chunker_list(seq, size):
    return [seq[i::size] for i in range(size)]

pd_snps_ids_chunks=chunker_list(PD_snps_ids_disgenet,4)

In [18]:
#applied this iteraltively for all 4 chunks of the snp ids cause of computative speed
a=0
with open ("DisGeNET_SNPIDs_3.txt","w") as DisGeNET_SNPIDs:
    for ID in pd_snps_ids_chunks[3]:
        a+=1
        DisGeNET_SNPIDs.write(ID+"\n")

80


In [36]:
# reading the chunks 
df_0=pd.read_csv("DisGeNET_SNPIDs_0.txt",delimiter = '\t')
df_1=pd.read_csv("DisGeNET_SNPIDs_1.txt",delimiter = '\t')
df_2=pd.read_csv("DisGeNET_SNPIDs_2.txt",delimiter = '\t')
df_3=pd.read_csv("DisGeNET_SNPIDs_3.txt",delimiter = '\t')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,8,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
#merging all dataframes
merged_df = pd.concat([df_0, df_1,df_2,df_3])

In [38]:
##keep column 6 and 25 (index 5 and 24). "SNP ID" and "Gene name"
merged_df=merged_df[['rsID','GENCODE_name']]

In [40]:
#drop duplicates
merged_df_unique=merged_df.drop_duplicates(keep="first",inplace=False)

In [41]:
merged_df

,rsID,GENCODE_name
0,rs7797870,KLHL7-AS1
1,rs6461685,KLHL7-AS1
2,rs12539467,KLHL7-AS1
3,rs6461687,KLHL7-AS1
4,rs6461688,KLHL7-AS1
...,...,...
18774,rs9963062,CTIF
18775,rs11661179,CTIF
18776,rs9964701,CTIF
18777,rs9964706,CTIF


In [42]:
#remove NaN valued rows and shady values 
merged_df_unique=merged_df_unique.dropna(inplace=False)
print("NaN values removed",len(merged_df_unique))

NaN values removed 6415


In [44]:
#make copy of mapping with legit name
Haploreg_DisGeNet_final=merged_df_unique
print (merged_df_unique)

             rsID GENCODE_name
0       rs7797870    KLHL7-AS1
1       rs6461685    KLHL7-AS1
2      rs12539467    KLHL7-AS1
3       rs6461687    KLHL7-AS1
4       rs6461688    KLHL7-AS1
...           ...          ...
18774   rs9963062         CTIF
18775  rs11661179         CTIF
18776   rs9964701         CTIF
18777   rs9964706         CTIF
18778   rs9952724         CTIF

[6415 rows x 2 columns]


In [ ]:
################## Query GTEx (gtex_v8) with PheWAS data #####################

In [45]:
#access Gtex portal thru API "https://gtexportal.org/home/api-docs/index.html#!/association/singleTissueEqtl"
#fetch only brain tissue qtls
with open('DisGeNet_snp_gene_eqtl.csv', 'a') as outfile: # save in csv
    for snp in PD_snps_ids_disgenet:
        command="curl -X GET --header \'Accept: text/html\' \'https://gtexportal.org/rest/v1/association/singleTissueEqtl?format=tsv&snpId="+snp+"&tissueSiteDetailId=Brain_Amygdala%2CBrain_Anterior_cingulate_cortex_BA24%2CBrain_Caudate_basal_ganglia%2CBrain_Cerebellar_Hemisphere%2CBrain_Cerebellum%2CBrain_Cortex%2CBrain_Frontal_Cortex_BA9%2CBrain_Hippocampus%2CBrain_Hypothalamus%2CBrain_Nucleus_accumbens_basal_ganglia%2CBrain_Putamen_basal_ganglia%2CBrain_Spinal_cord_cervical_c-1%2CBrain_Substantia_nigra&datasetId=gtex_v8\'"
        subprocess.call(command,shell=True,stdout=outfile)

In [46]:
eQTL_query_DisGeNet=pd.read_csv("DisGeNet_snp_gene_eqtl.csv",sep="\t")
#keep only snpID and gene symbol column
eQTL_query_DisGeNet=eQTL_query_DisGeNet[["snpId","geneSymbolUpper"]]
print (eQTL_query_DisGeNet)

            snpId  geneSymbolUpper
0      rs10256359        KLHL7-AS1
1      rs10256359            NUPL2
2      rs10256359      AC005082.12
3      rs10256359            GPNMB
4      rs10256359        KLHL7-AS1
...           ...              ...
15231   rs9841498        MCCC1-AS1
15232       snpId  geneSymbolUpper
15233       snpId  geneSymbolUpper
15234       snpId  geneSymbolUpper
15235       snpId  geneSymbolUpper

[15236 rows x 2 columns]


In [47]:
#drop duplicates
eQTL_query_DisGeNet=eQTL_query_DisGeNet.drop_duplicates(keep="first",inplace=False)
print ("No duplicates:",len(eQTL_query_DisGeNet))

No duplicates: 2323


In [55]:
eQTL_query_DisGeNet.head(10) #shady row in index 5

,snpId,geneSymbolUpper
0,rs10256359,KLHL7-AS1
1,rs10256359,NUPL2
2,rs10256359,AC005082.12
3,rs10256359,GPNMB
9,rs10256359,AK3P3
45,snpId,geneSymbolUpper
46,rs10445337,PLEKHM1
47,rs10445337,LRRC37A4P
48,rs10445337,DND1P1
49,rs10445337,MAPK8IP1P2


In [56]:
#drop shady row nr 5
eQTL_query_DisGeNet=eQTL_query_DisGeNet.drop(eQTL_query_DisGeNet.index[5])
print("final size",len(eQTL_query_DisGeNet))
print (eQTL_query_DisGeNet)

final size 2322
            snpId geneSymbolUpper
0      rs10256359       KLHL7-AS1
1      rs10256359           NUPL2
2      rs10256359     AC005082.12
3      rs10256359           GPNMB
9      rs10256359           AK3P3
...           ...             ...
15209    rs947211           RAB29
15214    rs947211           RAB7B
15220   rs9568188          CAB39L
15228   rs9841498           MCCC1
15229   rs9841498       MCCC1-AS1

[2322 rows x 2 columns]


In [57]:
#### combine Haploreg and GTEx results and only consider eQTL SNPs, which are in Haploreg results
#
#Drop snps that are not in Haploreg results
haplo_ids=Haploreg_DisGeNet_final.rsID.tolist()
eQTL_query_DisGeNet_haplofilter=eQTL_query_DisGeNet[eQTL_query_DisGeNet["snpId"].isin(haplo_ids)]

In [58]:
#renaming eQTL_query_PheWAS_haplofilter column names
eQTL_query_DisGeNet_haplofilter.columns=["rsID","GENCODE_name"]

In [59]:
#join Haploreg_PheWAS_final and eQTL_query_PheWAS_haplofilter
frames = [Haploreg_DisGeNet_final,eQTL_query_DisGeNet_haplofilter]
snps_gene_haploreg_gtex_DisGEnet = pd.concat(frames, keys=['haplo', 'gtex'])
#
snps_gene_haploreg_gtex_DisGEnet

rsID GENCODE_name
haplo 0       rs7797870    KLHL7-AS1
      1       rs6461685    KLHL7-AS1
      2      rs12539467    KLHL7-AS1
      3       rs6461687    KLHL7-AS1
      4       rs6461688    KLHL7-AS1
...                 ...          ...
gtex  15209    rs947211        RAB29
      15214    rs947211        RAB7B
      15220   rs9568188       CAB39L
      15228   rs9841498        MCCC1
      15229   rs9841498    MCCC1-AS1

[8599 rows x 2 columns]

In [60]:
#remove duplicates from joined dataframe
snps_gene_haploreg_gtex_DisGEnet=snps_gene_haploreg_gtex_DisGEnet.drop_duplicates(keep="first",inplace=False)

In [61]:
print(len(snps_gene_haploreg_gtex_DisGEnet)) #done
snps_gene_haploreg_gtex_DisGEnet.to_csv("snps_gene_haploreg_gtex_DisGEnet.csv",index=False)

8505


In [ ]:
######################################################